In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

Let's start with importing the required libraries and also suppressing any warnings
* keras - We will be using keras for building our models and also loading our data
* imager - This will be used mainly for plotting the images 

In [ ]:
suppressMessages(library(keras))
suppressMessages(library(imager))

# Global Constants

We will now define some constants that we are going to be using throughout the notebook such as training, validation and testing directories, image sizes and so on

* train_dir, validation_dir, test_dir - Directory paths in order to load the images
* image_size - As neural networks work with same size for all images, so we declare a fixed size for loading the images
* epochs - Shows the total number of epochs that the model will be trained
* output_classes - Shows the classes available here i.e, Normal and Pneumonia
* batch_size - Images will be loaded in batches in order to make sure that the RAM is utilized efficiently
* random_seed - In order to reproduce the results, we use the same random seed every time

In [ ]:
train_dir <- '../input/chest-xray-pneumonia/chest_xray/chest_xray/train/'
validation_dir <- '../input/chest-xray-pneumonia/chest_xray/chest_xray/val/'
test_dir <- '../input/chest-xray-pneumonia/chest_xray/chest_xray/test/'

epochs <- 100
image_size <- c(224, 224)
random_seed <- 123
output_classes <- c('NORMAL', 'PNEUMONIA')
batch_size <- 32

# Problem Understanding

Let's start with first defining the problem in hand:

> We want to build a Machine Learning model to classify x-ray images into 'NORMAL' and 'PNEUMONIA' classes

As we can see here that this is a classification problem and infact a binary classification problem. And we know that Convolutional Neural Networks (CNNs) are best at capturing various features in the images and can thus be good at classifying.

So now we know about the problem and the machine learning algorithm that we are going to use, we can look deeper into the data and then start working with different configurations of CNN to choose the best model

# Data Understanding (Exploratory Data Analysis)

Lets dive deeper into the data. First lets see how many images are there in each class in respective training, validation and testing sets

In [ ]:
train_normal_images <- list.files(paste0(train_dir, output_classes[1]))
train_pneumonia_images <- list.files(paste0(train_dir, output_classes[2]))

validation_normal_images <- list.files(paste0(validation_dir, output_classes[1]))
validation_pneumonia_images <- list.files(paste0(validation_dir, output_classes[2]))

test_normal_images <- list.files(paste0(test_dir, output_classes[1]))
test_pneumonia_images <- list.files(paste0(test_dir, output_classes[2]))

total_normal_images <- c(train_normal_images, validation_normal_images, test_normal_images)
total_pneumonia_images <- c(train_pneumonia_images, validation_pneumonia_images, test_pneumonia_images)

total_train_images <- c(train_normal_images, train_pneumonia_images)
total_validation_images <- c(validation_normal_images, validation_pneumonia_images)
total_test_images <- c(test_normal_images, test_pneumonia_images)

total_images <- c(total_normal_images, total_pneumonia_images)

In [ ]:
print(paste0('Total number of images: ', length(total_images)))
print(paste0('Total number of Training images: ', length(total_train_images)))
print(paste0('Total number of Validation images: ', length(total_validation_images)))
print(paste0('Total number of Testing images: ', length(total_test_images)))

print(paste0('Total number of Normal images: ', length(total_normal_images)))
print(paste0('Total number of Normal Training images: ', length(train_normal_images)))
print(paste0('Total number of Normal Validation images: ', length(validation_normal_images)))
print(paste0('Total number of Normal Testing images: ', length(test_normal_images)))

print(paste0('Total number of Pneumonia images: ', length(total_pneumonia_images)))
print(paste0('Total number of Pneumonia Training images: ', length(train_pneumonia_images)))
print(paste0('Total number of Pneumonia Validation images: ', length(validation_pneumonia_images)))
print(paste0('Total number of Pneumonia Testing images: ', length(test_pneumonia_images)))



Lets also construct bar charts to show these values

In [ ]:
total <- c(length(total_images), length(total_train_images), length(total_test_images), length(total_validation_images))
total_args <- c('Total', 'Training', 'Testing', 'Validation')

normal <- c(length(total_normal_images), length(train_normal_images), length(test_normal_images), length(validation_normal_images))
normal_args <- c('Total', 'Training', 'Testing', 'Validation')

pneumonia <- c(length(total_pneumonia_images), length(train_pneumonia_images), length(test_pneumonia_images), length(validation_pneumonia_images))
pneumonia_args <- c('Total', 'Training', 'Testing', 'Validation')


barplot(total, names.arg=total_args, xlab='Distribution', ylab='Number of Images', main='Distribution of Total images')
barplot(normal, names.arg=normal_args, xlab='Distribution', ylab='Number of Images', main='Distribution of Normal images')
barplot(pneumonia, names.arg=pneumonia_args, xlab='Distribution', ylab='Number of Images', main='Distribution of Pneumonia images')

Lets visualize some images belonging to each class. For this, we will write a helper function

In [ ]:
normal_train_folder <- '../input/chest-xray-pneumonia/chest_xray/chest_xray/train/NORMAL/'
pneumonia_train_folder <- '../input/chest-xray-pneumonia/chest_xray/chest_xray/train/PNEUMONIA/'

plot_images <- function(folder, images) {
    par(mfrow=c(2,2))
    
    for(i in 1:4) {
        image <- load.image(paste0(folder, images[i]))
        plot(image)
    }
}

**Normal Images**

In [ ]:
plot_images(normal_train_folder, train_normal_images)

**Pneumonia Images**

In [ ]:
plot_images(pneumonia_train_folder, train_pneumonia_images)

# Data Preparation

Now before we start modeling and training the models on the data, we need to prepare the data to be ready for training. For this, we will use `image_data_generator` from `keras` library to rescale the images on the fly. Rescaling helps in making the training process faster. We will start with just rescaling and then add augmentation to understand how it improves our evaluation

The way the `image_data_generator` works is if it has a folder that contains images in different folders with the classes names. For instance, there will be a 'NORMAL' and a 'PNEUMONIA' folder inside the train folder. It will load images batch wise and assign labels according to its parent folder. It will also apply rescaling as it loads the images

In [ ]:
train_datagen = image_data_generator(rescale = 1/255,)
validation_datagen <- image_data_generator(rescale = 1/255)  
test_datagen <- image_data_generator(rescale = 1/255)  

train_generator <- flow_images_from_directory(
  train_dir,                            
  train_datagen,                        
  classes = output_classes,
  target_size = image_size,            
  batch_size = batch_size,
  class_mode = "categorical",
  shuffle = TRUE,
  seed = random_seed
)

validation_generator <- flow_images_from_directory(
  validation_dir,
  classes = output_classes,
  validation_datagen,
  target_size = image_size,
  batch_size = batch_size,
  class_mode = "categorical",
  shuffle = TRUE,
  seed = random_seed
)

test_generator <- flow_images_from_directory(
  test_dir,
  classes = output_classes,
  test_datagen,
  target_size = image_size,
  batch_size = 1,
  class_mode = "categorical",
  shuffle = FALSE
)

# Modelling

Now comes the most interesting part, *the modelling and training part*. We will start with the most basic models and then go onto adding more complex models such as with transfer learning

### Model 1: CNN from Scratch 

**Model Building**

Our first model will be building a CNN from scratch with no augmentation or transfer learning

Any CNN consists of a sequence of convolutional and pooling layers followed by around a couple of fully connected layers at the end. As we go forward, the height and width of the image decreases but the depth increases. For instance, at the start the input size will be (224, 224, 3) but after the first convolutional layer, the depth increases to 32 (filter size) and the max pooling layer reduces the width and height by a factor of 2

Finally, the image is flattened (like a vector) and then fed into fully connected layers. It outputs 2 neurons at last which shows probability of the image being one of the output classes (Normal and Pneumonia).

The activation functions used here are 'relu' and 'softmax'. The intermediate layers use 'relu' which is just the function max(x, 0) which means take the value if its positive otherwise 0. The 'softmax' function on the otherhand calculates the probability of each neuron and that's why its used at the last layer

In [ ]:
model_scratch <- keras_model_sequential() %>% 
  layer_conv_2d(filters = 32, kernel_size = c(3,3), activation = "relu", 
                input_shape = c(224,224,3)) %>% 
  layer_max_pooling_2d(pool_size = c(2,2)) %>% 
  layer_conv_2d(filters = 64, kernel_size = c(3,3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2,2)) %>% 
  layer_conv_2d(filters = 64, kernel_size = c(3,3), activation = "relu") %>%
  layer_max_pooling_2d(pool_size = c(2,2)) %>% 
  layer_flatten() %>% 
  layer_dense(units = 64, activation = "relu") %>% 
  layer_dense(units = 2, activation = "softmax")

Let's analyze the summary of the model

In [ ]:
summary(model_scratch)

Let's compile the model with loss and optimizer. Finally we will train the model

**Model Training**

First, we have to compile the model with appropriate loss, optimizer and metrics. The loss function used here is 'categorical_crossentropy' because there are multiple classes but each image belongs to a single class. For multi-label classification, you would use 'binary_crossentropy'. The optimizer used here is 'rmsprop' as it helps with faster learning. You can use Adam too. In the metrics, I have only included 'accuracy' but you can add more like loss, f1 score etc.

In [ ]:
set.seed(123)

model_scratch %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_rmsprop(lr = 1e-6),
  metrics = c("accuracy")
)

validation_steps <- ceiling(length(total_validation_images) / batch_size)

history <- model_scratch %>% fit_generator(
  train_generator,
  steps_per_epoch = 100,
  epochs = epochs,
  validation_data = validation_generator,
  validation_steps = validation_steps,
  verbose = 1
)

Let's plot the metrics now

In [ ]:
plot(history)

We will now evaluate the model on the test set

**Model Evaluation**

In [ ]:
preds = evaluate_generator(model_scratch,
                          test_generator,
                          steps = length(total_test_images))
preds

So we have achieved an accuracy of **75.6** using our CNN model from scratch. Let's use transfer learning now

### Model 2: Transfer Learning 

**Model Building**

In the previous section, we had to train our model from scratch which not only takes much longer but is not deep enough to understand the features in the image. For this purpose, we will use Transfer Learning. Transfer Learning is basically taking a model that somebody has trained already for hundreds of epochs on millions of images and attach your own model to it for faster and more efficient training. 

We will start with an XceptionNet base and add our custom network to identify pneumonia in X-ray images. You can read more about XceptionNet [here](https://arxiv.org/abs/1610.02357)

In [ ]:
conv_base <- application_xception(
  weights = "imagenet",
  include_top = FALSE,
  input_shape = c(224, 224, 3)
)


In [ ]:
summary(conv_base)

Let's add our custom network and compile the model. We have to add our custom network as we have less output classes as compared to `imagenet` which contains 1000 classes.

In [ ]:
model_xception <- keras_model_sequential() %>% 
  conv_base %>% 
  layer_global_average_pooling_2d(trainable = T) %>%
  layer_dense(units = 1024, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.3, trainable = T) %>%
  layer_dense(units = 512, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.2, trainable = T) %>%
  layer_dense(units = 256, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.2, trainable = T) %>%
  layer_dense(units = 2, activation = "softmax", trainable = T)

#Don't train the base,
freeze_weights(conv_base)

set.seed(123)

model_xception %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_rmsprop(lr = 1e-6),
  metrics = c("accuracy")
)

**Model Training**

In [ ]:
history <- model_xception %>% fit_generator(
  train_generator,
  steps_per_epoch = 100,
  epochs = epochs,
  validation_data = validation_generator,
  validation_steps = validation_steps,
  verbose = 1
)

In [ ]:
plot(history)

**Model Evaluation**

In [ ]:
preds = evaluate_generator(model_xception,
                          test_generator,
                          steps = length(total_test_images))
preds

So we have achieved an accuracy of **78.6** using our Transfer Learning model (XceptionNet). 

### Model 3: Transfer Learning with Augmentation

**Model Building**

Here, we would use the same model as above (Xception) but will change our `train_generator` so that it applies data augmentation i.e, rotation, zoom, shift etc. This allows the model to classify any image currently irrespective of its position in the overall image.

This is especially required because the real world images are from different angles such as zoomed in, rotated or the subject of the image is at different parts of the image. As a human, we can understand this but for machines, they deal with pixel values and thus for them, its completely different. That's why we will be using data augmentation to help tackle this issue. We are using the Transfer Learning model as it performed better than the model we trained from scratch


We don't do it for the validation or test set because we want to predict on those and hence we use them as such

In [ ]:
train_datagen = image_data_generator(
  rescale = 1/255,
  rotation_range = 5,
  width_shift_range = 0.1,
  height_shift_range = 0.05,
  shear_range = 0.1,
  zoom_range = 0.15,
  horizontal_flip = TRUE,
  vertical_flip = FALSE,
  fill_mode = "reflect"
)

train_generator <- flow_images_from_directory(
  train_dir,                            
  train_datagen,                        
  classes = output_classes,
  target_size = image_size,            
  batch_size = batch_size,
  class_mode = "categorical",
  shuffle = TRUE,
  seed = random_seed
)

conv_base <- application_xception(
  weights = "imagenet",
  include_top = FALSE,
  input_shape = c(224, 224, 3)
)


model_augmentation <- keras_model_sequential() %>% 
  conv_base %>% 
  layer_global_average_pooling_2d(trainable = T) %>%
  layer_dense(units = 1024, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.3, trainable = T) %>%
  layer_dense(units = 512, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.2, trainable = T) %>%
  layer_dense(units = 256, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.2, trainable = T) %>%
  layer_dense(units = 2, activation = "softmax", trainable = T)

#Don't train the base,
freeze_weights(conv_base)

set.seed(123)

model_augmentation %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_rmsprop(lr = 1e-6),
  metrics = c("accuracy")
)


**Model Training**

In [ ]:
history <- model_augmentation %>% fit_generator(
  train_generator,
  steps_per_epoch = 100,
  epochs = epochs,
  validation_data = validation_generator,
  validation_steps = validation_steps,
  verbose = 1
)

In [ ]:
plot(history)

**Model Evaluation**

In [ ]:
preds = evaluate_generator(model_augmentation,
                          test_generator,
                          steps = length(total_test_images))
preds

So we have achieved an accuracy of **84.6** using our Transfer Learning model (XceptionNet) with Data Augmentation. 

### Model 4: Using `class_weight` to counter class imbalance

As we saw earlier, that the number of images in NORMAL class are much lower than the images in PNEUMONIA class. We can use `class_weight` attribute during training to ensure that the loss function takes this class imbalance into account when penalizing the wrong predictions.

The model building part will be the same as the last section

**Model Building**

In [ ]:
conv_base <- application_xception(
  weights = "imagenet",
  include_top = FALSE,
  input_shape = c(224, 224, 3)
)


model_class_weight <- keras_model_sequential() %>% 
  conv_base %>% 
  layer_global_average_pooling_2d(trainable = T) %>%
  layer_dense(units = 1024, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.3, trainable = T) %>%
  layer_dense(units = 512, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.2, trainable = T) %>%
  layer_dense(units = 256, activation = "relu", trainable = T) %>% 
  layer_dropout(rate = 0.2, trainable = T) %>%
  layer_dense(units = 2, activation = "softmax", trainable = T)

#Don't train the base,
freeze_weights(conv_base)

set.seed(123)

model_class_weight %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_rmsprop(lr = 1e-6),
  metrics = c("accuracy")
)



**Model Training**

In training, we will use the `class_weight` argument. We will calculate it as follows:

In [ ]:
class_weight <- list(
    '0'<- length(total_images) / length(total_normal_images),
    '1'<- 1.0
)
class_weight

This means for every 1 example of PNEUMONIA, we will consider around 3.7 examples of NORMAL images

In [ ]:
history <- model_class_weight %>% fit_generator(
  train_generator,
  steps_per_epoch = 100,
  epochs = epochs,
  validation_data = validation_generator,
  validation_steps = validation_steps,
  verbose = 1,
  class_weight = class_weight
)

In [ ]:
plot(history)

**Model Evaluation**

In [ ]:
preds = evaluate_generator(model_class_weight,
                          test_generator,
                          steps = length(total_test_images))
preds

So we have achieved an accuracy of **85.2** using our Transfer Learning model (XceptionNet) with Data Augmentation and `class_weight` inclusion. 